In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [29]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import helium

plotdir = '../../cvxportfolio/portfolio/plots/'
datadir='../../cvxportfolio/data/'

sigmas=pd.read_csv(datadir+'sigmas.csv.gz',index_col=0,parse_dates=[0]).iloc[:,:-1]
returns=pd.read_csv(datadir+'returns.csv.gz',index_col=0,parse_dates=[0])
volumes=pd.read_csv(datadir+'volumes.csv.gz',index_col=0,parse_dates=[0]).iloc[:,:-1]


subset_tickers = ['AMZN', 'GOOGL', 'TSLA', 'NKE', 'IBM', 'BAC', 'C', 'USDOLLAR']
sigmas = sigmas.ix[:, sigmas.columns.isin(subset_tickers)]
returns = returns.ix[:, returns.columns.isin(subset_tickers)]
volumes = volumes.ix[:, volumes.columns.isin(subset_tickers)]

start_date="2012-01-01"
end_date="2016-12-31"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [30]:
cash_ticker = "_CASH"

In [31]:
sigmas[cash_ticker] = 0.
returns = returns.rename(columns={"USDOLLAR": cash_ticker})
volumes[cash_ticker] = np.finfo(np.float).max

In [32]:
w_b = pd.Series(index=returns.columns, data=1)
w_b.USDOLLAR = 0.
w_b/=sum(w_b)


In [33]:
return_estimate=pd.read_csv(datadir+'return_estimate.csv.gz',index_col=0,parse_dates=[0]).dropna()
volume_estimate=pd.read_csv(datadir+'volume_estimate.csv.gz',index_col=0,parse_dates=[0]).iloc[:,:-1].dropna()
sigma_estimate=pd.read_csv(datadir+'sigma_estimate.csv.gz',index_col=0,parse_dates=[0]).iloc[:,:-1].dropna()

return_estimate = return_estimate.ix[:, return_estimate.columns.isin(subset_tickers)]
volume_estimate = volume_estimate.ix[:, volume_estimate.columns.isin(subset_tickers)]
sigma_estimate = sigma_estimate.ix[:, sigma_estimate.columns.isin(subset_tickers)]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [34]:
sigma_estimate[cash_ticker] = 0.
return_estimate = return_estimate.rename(columns={"USDOLLAR": cash_ticker})
volume_estimate[cash_ticker] = np.finfo(np.float).max

In [41]:
borrow_costs = returns.copy()
borrow_costs[:] = 1E-4
borrow_costs[cash_ticker] = 0.

dividends = returns.copy()
dividends[:] = 0.

simulated_hcost=helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)
simulated_tcost=helium.TransactionCost(gamma =1.0, 
                                             half_spread=0.0005/2,
                                             nonlin_coef = 1.,
                                             sigmas = sigmas, 
                                             nonlin_power = 1.5,
                                             volumes = volumes,
                                             asym_coef = 0.)

simulator = helium.MarketSimulator(returns, volumes, costs=[simulated_tcost, simulated_hcost], cash_ticker=cash_ticker)

In [42]:
deltas = return_estimate.copy()
deltas[:]  = 0.
r= helium.DefaultRet(return_estimate, deltas = deltas, gamma_decay=None)
Sigma_hat = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack().dropna()

In [44]:
policies = {}
results = {}
gamma_risks_coarse=[.1,.3,1,3,10,30,100,300,1000]
gamma_tcosts_coarse=[1,2,5,10,20]
for gamma_risk in gamma_risks_coarse:
    for gamma_tcost in gamma_tcosts_coarse: 
        risk_model = helium.BasicRiskCost(5.0, Sigma_hat)
        optimization_hcost = helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)
        optimization_tcost = helium.TransactionCost(gamma_tcost, 
                                                    half_spread=0.0005/2., 
                                                    nonlin_coef = 1.,
                                                    sigmas = sigma_estimate, 
                                                    nonlin_power = 1.5,
                                                    volumes = volume_estimate,
                                                    asym_coef = 0.)
        
        # issue optimization_tcost
        costs = [risk_model, optimization_tcost, optimization_hcost]
        const = [helium.LeverageLimitConstraint(3)]
        policies[(gamma_risk, gamma_tcost)] = helium.SinglePeriodOpt(r, costs, const)
        results[(gamma_risk, gamma_tcost)] = simulator.run(1E8*w_b, 
                                                         policies[(gamma_risk, gamma_tcost)],
                                                         start_date, 
                                                         end_date)
        break
    break

INFO:root:Backtest started, from 2012-01-03 00:00:00 to 2016-12-30 00:00:00
INFO:root:Getting trades at date: 2012-01-03 00:00:00
INFO:root:Propagating portfolio at time 2012-01-03 00:00:00
INFO:root:Getting trades at date: 2012-01-04 00:00:00
INFO:root:Propagating portfolio at time 2012-01-04 00:00:00
INFO:root:Getting trades at date: 2012-01-05 00:00:00
INFO:root:Propagating portfolio at time 2012-01-05 00:00:00
INFO:root:Getting trades at date: 2012-01-06 00:00:00
INFO:root:Propagating portfolio at time 2012-01-06 00:00:00


******
h:GOOGL    1.428571e+07
AMZN     1.428571e+07
BAC      1.428571e+07
C        1.428571e+07
IBM      1.428571e+07
NKE      1.428571e+07
_CASH    1.428571e+07
dtype: float64
GOOGL    0.005513
AMZN     0.000581
BAC     -0.000355
C       -0.000601
IBM     -0.009568
NKE     -0.031428
_CASH    0.035856
dtype: float64
******
******
h:GOOGL    1.528529e+07
AMZN     1.483526e+07
BAC      1.486541e+07
C        1.531791e+07
IBM      1.350434e+07
NKE      1.119380e+07
_CASH    1.786787e+07
dtype: float64
GOOGL   -0.029833
AMZN     0.006378
BAC     -0.012102
C       -0.001621
IBM     -0.138449
NKE      0.000208
_CASH    0.175418
dtype: float64
******
******
h:GOOGL    1.226911e+07
AMZN     1.535987e+07
BAC      1.364398e+07
C        1.506560e+07
IBM     -7.349174e+05
NKE      1.137854e+07
_CASH    3.590642e+07
dtype: float64
GOOGL   -0.141236
AMZN     0.000002
BAC     -0.000881
C        0.000203
IBM     -0.010312
NKE      0.006607
_CASH    0.145617
dtype: float64
******
******
h:GOOGL   -2.23

INFO:root:Getting trades at date: 2012-01-09 00:00:00
INFO:root:Propagating portfolio at time 2012-01-09 00:00:00
INFO:root:Getting trades at date: 2012-01-10 00:00:00
INFO:root:Propagating portfolio at time 2012-01-10 00:00:00
INFO:root:Getting trades at date: 2012-01-11 00:00:00
INFO:root:Propagating portfolio at time 2012-01-11 00:00:00
INFO:root:Getting trades at date: 2012-01-12 00:00:00
INFO:root:Propagating portfolio at time 2012-01-12 00:00:00
INFO:root:Getting trades at date: 2012-01-13 00:00:00
INFO:root:Propagating portfolio at time 2012-01-13 00:00:00
INFO:root:Getting trades at date: 2012-01-17 00:00:00


******
h:GOOGL   -2.220425e+06
AMZN     1.531708e+07
BAC      1.439237e+07
C        1.456247e+07
IBM     -4.306948e+06
NKE      1.523071e+07
_CASH    5.147963e+07
dtype: float64
GOOGL   -0.003137
AMZN    -0.002923
BAC     -0.000559
C       -0.014559
IBM     -0.017590
NKE      0.007476
_CASH    0.031292
dtype: float64
******
******
h:GOOGL   -2.440097e+06
AMZN     1.467884e+07
BAC      1.454268e+07
C        1.328371e+07
IBM     -6.112371e+06
NKE      1.607048e+07
_CASH    5.474483e+07
dtype: float64
GOOGL    0.002384
AMZN     0.002128
BAC     -0.013652
C       -0.004040
IBM     -0.103993
NKE     -0.009455
_CASH    0.126628
dtype: float64
******
******
h:GOOGL   -2.192700e+06
AMZN     1.496692e+07
BAC      1.386527e+07
C        1.326736e+07
IBM     -1.698132e+07
NKE      1.509830e+07
_CASH    6.799122e+07
dtype: float64
GOOGL   -8.505235e-07
AMZN     1.287684e-06
BAC      1.381483e-05
C        7.193279e-04
IBM     -9.704271e-02
NKE     -8.376050e-02
_CASH    1.800696e-01
dtype: float64
*

INFO:root:Propagating portfolio at time 2012-01-17 00:00:00
INFO:root:Getting trades at date: 2012-01-18 00:00:00
INFO:root:Propagating portfolio at time 2012-01-18 00:00:00
INFO:root:Getting trades at date: 2012-01-19 00:00:00
INFO:root:Propagating portfolio at time 2012-01-19 00:00:00
INFO:root:Getting trades at date: 2012-01-20 00:00:00


...

GOOGL    0.021383
AMZN    -0.003665
BAC     -0.021253
C        0.005486
IBM     -0.055682
NKE     -0.009477
_CASH    0.063208
dtype: float64
******
******
h:GOOGL    5.711143e+07
AMZN    -4.070258e+07
BAC     -1.172824e+07
C        5.893449e+07
IBM     -3.483912e+07
NKE     -3.304094e+07
_CASH    1.378035e+08
dtype: float64
GOOGL   -0.004430
AMZN     0.007645
BAC     -0.024369
C        0.022429
IBM      0.000060
NKE     -0.067724
_CASH    0.066389
dtype: float64
******
******
h:GOOGL    5.670548e+07
AMZN    -4.011264e+07
BAC     -1.486344e+07
C        6.183642e+07
IBM     -3.482069e+07
NKE     -4.202680e+07
_CASH    1.466518e+08
dtype: float64
GOOGL   -0.081214
AMZN     0.041771
BAC      0.017892
C       -0.000290
IBM      0.000436
NKE      0.001325
_CASH    0.020080
dtype: float64
******
******
h:GOOGL    4.602634e+07
AMZN    -3.477707e+07
BAC     -1.260480e+07
C        6.297872e+07
IBM     -3.495437e+07
NKE     -4.262388e+07
_CASH    1.493121e+08
dtype: float64


INFO:root:Propagating portfolio at time 2016-12-21 00:00:00
INFO:root:Getting trades at date: 2016-12-22 00:00:00
INFO:root:Propagating portfolio at time 2016-12-22 00:00:00
INFO:root:Getting trades at date: 2016-12-23 00:00:00
INFO:root:Propagating portfolio at time 2016-12-23 00:00:00
INFO:root:Getting trades at date: 2016-12-27 00:00:00


GOOGL   -0.023682
AMZN    -0.000338
BAC      0.003794
C        0.001358
IBM      0.001741
NKE     -0.020675
_CASH    0.037801
dtype: float64
******
******
h:GOOGL    4.271048e+07
AMZN    -3.479408e+07
BAC     -1.205628e+07
C        6.310787e+07
IBM     -3.466623e+07
NKE     -4.582791e+07
_CASH    1.543396e+08
dtype: float64
GOOGL   -3.400363e-04
AMZN     6.298309e-02
BAC      4.629020e-03
C       -6.825276e-10
IBM      8.066374e-02
NKE      1.087187e-03
_CASH   -1.490230e-01
dtype: float64
******
******
h:GOOGL    4.253293e+07
AMZN    -2.628298e+07
BAC     -1.139598e+07
C        6.283776e+07
IBM     -2.391434e+07
NKE     -4.554377e+07
_CASH    1.345300e+08
dtype: float64
GOOGL   -8.074382e-05
AMZN    -6.338702e-02
BAC      6.493937e-03
C       -4.680110e-03
IBM      1.406982e-08
NKE      1.088606e-02
_CASH    5.076787e-02
dtype: float64
******
******
h:GOOGL    4.242347e+07
AMZN    -3.443813e+07
BAC     -1.056186e+07
C        6.273070e+07
IBM     -2.386424e+07
NKE     -4.390398e+07
_CA

INFO:root:Propagating portfolio at time 2016-12-27 00:00:00
INFO:root:Getting trades at date: 2016-12-28 00:00:00
INFO:root:Propagating portfolio at time 2016-12-28 00:00:00
INFO:root:Getting trades at date: 2016-12-29 00:00:00
INFO:root:Propagating portfolio at time 2016-12-29 00:00:00
INFO:root:Getting trades at date: 2016-12-30 00:00:00
INFO:root:Propagating portfolio at time 2016-12-30 00:00:00


GOOGL   -4.511008e-02
AMZN     4.089165e-09
BAC     -1.515516e-02
C       -4.926784e-04
IBM      2.980462e-07
NKE      9.477362e-03
_CASH    5.128026e-02
dtype: float64
******
******
h:GOOGL    3.649088e+07
AMZN    -3.492750e+07
BAC     -1.259279e+07
C        6.276762e+07
IBM     -2.392574e+07
NKE     -4.212832e+07
_CASH    1.480958e+08
dtype: float64
GOOGL    1.047990e-02
AMZN     1.187318e-02
BAC     -5.161771e-04
C        3.112221e-02
IBM     -1.061058e-01
NKE      2.011407e-08
_CASH    5.314666e-02
dtype: float64
******
******
h:GOOGL    3.764211e+07
AMZN    -3.337065e+07
BAC     -1.250509e+07
C        6.578061e+07
IBM     -3.790389e+07
NKE     -4.190656e+07
_CASH    1.551651e+08
dtype: float64
GOOGL    0.076947
AMZN     0.037644
BAC      0.000205
C       -0.034343
IBM      0.023260
NKE      0.037018
_CASH   -0.140732
dtype: float64
******
******
h:GOOGL    4.776798e+07
AMZN    -2.811132e+07
BAC     -1.229339e+07
C        6.054363e+07
IBM     -3.489847e+07
NKE     -3.701579e+07
_CA

In [48]:
results[(0.1, 1)]

In [11]:
result_df_coarse=pd.DataFrame()
for k in results:
    if k[0] in gamma_risks_coarse and k[1] in gamma_tcosts_coarse:
        result_df_coarse.loc[k[0], k[1]] = results[k]
        
result_df = result_df_coarse.loc[sorted(result_df_coarse.index), sorted(result_df_coarse.columns)]

In [12]:
plt.figure(figsize=(8,5))
for gamma_tcost in result_df.columns:
    x=[el.excess_returns.std()*100*np.sqrt(250) for el in result_df[gamma_tcost]]
    y=[el.excess_returns.mean()*100*250 for el in result_df[gamma_tcost]]
    plt.plot(np.array(x),np.array(y), '.-', label='$\gamma^\mathrm{trade} = %g$'%gamma_tcost)
plt.legend(loc='lower right')
plt.xlabel('Risk')
plt.ylabel('Return')
plt.xlim([0,20])
plt.ylim([0,30])


import matplotlib.ticker as mtick
ax = plt.gca()
ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f%%'))
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f%%'))

#plt.savefig(plotdir+'spo_riskrewardfrontier.png')